## Generate data and appply classical hedging
## Perform evaluation

In [9]:
from simulation import *
import numpy as np
from scipy.stats import norm
from tqdm import tqdm
import matplotlib.pyplot as plt

In [22]:
T = 60 # length of simulation 20=1 month, 60 = three months
S0 = 100 # starting price
K = 100 # strike price
sigma = 0.2 # volatility
r = 0 # risk-free rate
q = 0 # dividend yield
mu = 0.05 # expected return on stock
kappa = 0.01 # trading cost per unit traded
dt = 1 # hedging time step
notional = 100 # how many stocks the option is on
rho = -0.4 # correlation of stochastic volatility process
v = 0.6
sigma0 = 0.2 # starting volatility
c = 1.5 #standard deviation coefficient
ds = 0.01

n = 25000 # number of simulatied paths
days = 250 # number of days in a year
freq = 1 # trading frequency
np.random.seed(1)

In [23]:
S_gbm, p_gbm, d_gbm = simulateGBM(n, T, dt, S0, mu, r, q, sigma, days, freq, K)

S_sabr, s_sabr, iv_sabr, p_sabr, delta_sabr, delta_sabr_bl = simulateSABR(
    n, T, dt, S0, mu, r, q, sigma, days, freq, rho, ds, v, K)

100%|██████████| 59/59 [00:00<00:00, 177.64it/s]


In [24]:
# classical hedging implementations (trading strategies)
trading_gbm, holding_gbm = hedgingStrategy("GBM",notional, d_gbm, 0)
trading_sabr, holding_sabr, trading_sabr_bl, holding_sabr_bl = hedgingStrategy("SABR",notional, delta_sabr, delta_sabr_bl)

In [25]:
# Accounting PnL
APL_gbm, holding_lagged_gbm = APL_process(S_gbm, p_gbm*notional, holding_gbm, K, notional, kappa)
APL_sabr, holding_lagged_sabr = APL_process(S_sabr, p_sabr*notional, holding_sabr, K, notional, kappa)
APL_sabr_bl, holding_lagged_sabr_bl = APL_process(S_sabr, p_sabr*notional, holding_sabr_bl, K, notional, kappa)

In [26]:
# Evaluation:
# Y(0) (take mean), 
# mean of  cost as percentage of option price,
# std of  cost as percentage of option price.

Y_gbm, mPerc_gbm, stdPerc_gbm = evaluate(APL_gbm, p_gbm, c, notional) 
Y_sabr, mPerc_sabr, stdPerc_sabr = evaluate(APL_sabr, p_sabr, c, notional)
Y_sabr_bl, mPerc_sabr_bl, stdPerc_sabr_bl = evaluate(APL_sabr_bl, p_sabr, c, notional)

In [29]:
print("GBM")
print("Y0: ", np.mean(Y_gbm))
print("mean cost (%): ", mPerc_gbm)
print("std cost (%): ", stdPerc_gbm)
print("\n")
print("SABR delta")
print("Y0: ", np.mean(Y_sabr))
print("mean cost (%): ", mPerc_sabr)
print("std cost (%): ", stdPerc_sabr)
print("\n")
print("SABR Bartlett")
print("Y0: ", np.mean(Y_sabr_bl))
print("mean cost (%): ", mPerc_sabr_bl)
print("std cost (%): ", stdPerc_sabr_bl)

GBM
Y0:  233.75656209909417
mean cost (%):  0.3118939779877127
std cost (%):  0.19091238252812498


SABR delta
Y0:  236.65688202375188
mean cost (%):  0.3114695347126298
std cost (%):  0.1961439481746562


SABR Bartlett
Y0:  237.37367164229065
mean cost (%):  0.31337575787764466
std cost (%):  0.1960961383585505


In [ ]:
# define data
# save to csv file
np.savetxt('asset_price_GBM_sim.csv', S_gbm, delimiter=',')
np.savetxt('option_price_GBM_sim.csv', p_gbm*notional, delimiter=',')
np.savetxt('asset_price_SABR_sim.csv', S_sabr, delimiter=',')
np.savetxt('option_price_SABR_sim.csv', p_sabr*notional, delimiter=',')


S_mixed = np.vstack((S_gbm, S_sabr))
p_mixed = np.vstack((p_gbm*notional, p_sabr*notional))
print(S_mixed.shape)
print(p_mixed.shape)
np.savetxt("asset_price_mixed_sim.csv", S_mixed, delimiter=",")
np.savetxt("option_price_mixed_sim.csv", p_mixed, delimiter=",")
